# Consulta a ChatGPT usando tu mano y tu micrófono

**Al correr el código, se abrirá una la cámara y podrás cambiar distintas funciones:**

1) Cambia el valor de 'reconocer_manos' para que reconocer las manos en el video.

2) Cambia el valor de 'usar_numeros' para mostrar las 21 marcas en la mano.

3) Usa 'cambiar_color' para cambiar el color de algunas marcas de la mano. Se debe ingresar como lista con valores entre 0 a 20, por ejemplo; \[2, 5, 8, 17\]

4) Cambia 'color' para cambiar el color de las marcas señaladas. Los valores son en escala BGR (Azul, Verde, Rojo) desde 0 a 255.

5) Usa 'pregunta' para realizar una pregunta a chatGPT cuando tengas las manos cerca de la pantalla. Una vez realizada la pregunta, vuelve a alejarte.

In [15]:
# ----------------------------------------------
# Read libraries
# ----------------------------------------------
import cv2
import time
import mediapipe as mp
import numpy as np
import openai
import speech_recognition as sr
import mediapipe as mp
import pyttsx3

from ultralytics import YOLO
from ultralytics.models.yolo.detect.predict import DetectionPredictor

import Codes

# ----------------------------------------------
# Inicializadores
# ----------------------------------------------

# Configura tu clave de API de OpenAI aquí
openai.api_key = "YOUR API KEY"
# Inicializa el reconocedor de voz
recognizer = sr.Recognizer()
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
# Initialize MediaPipe Drawing
mp_drawing = mp.solutions.drawing_utils

# Inicializar el sintetizador de voz
engine = pyttsx3.init()
engine.setProperty('rate', 180)

# ----------------------------------------------
# Hand recognizer implemented with chat GPT
# ----------------------------------------------
def handTracking(show, number, changeColor, color, question):
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue
        # Convert the BGR image to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Process the frame to detect hands
        results = hands.process(frame_rgb)
    
        if results.multi_hand_landmarks:
            # Determinar el centro de la imagen
            center_x = frame.shape[1] // 2
            
            for i, landmarks in enumerate(results.multi_hand_landmarks):
                # Calcular la posición horizontal del centro de la mano
                x_min = min(landmark.x for landmark in landmarks.landmark)
                x_max = max(landmark.x for landmark in landmarks.landmark)
                hand_center_x = (x_min + x_max) / 2 * frame.shape[1]
                
                # Determinar la mano izquierda o derecha
                if hand_center_x < center_x:
                    hand_type = "Mano derecha"
                else:
                    hand_type = "Mano Izquierda"
                
                # Draw landmarks on the frame
                mp_drawing.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)
    
                # Obtener las coordenadas del bounding box
                x_min = min(landmark.x for landmark in landmarks.landmark)
                y_min = min(landmark.y for landmark in landmarks.landmark)
                x_max = max(landmark.x for landmark in landmarks.landmark)
                y_max = max(landmark.y for landmark in landmarks.landmark)
                
                # Dibujar el cuadro alrededor de la mano
                cv2.rectangle(frame, (int(x_min * frame.shape[1]), int(y_min * frame.shape[0])),
                              (int(x_max * frame.shape[1]), int(y_max * frame.shape[0])),
                              (0, 255, 0), 2)
                
                # Agregar texto con el tipo de mano
                cv2.putText(frame, hand_type, (int(x_min * frame.shape[1]), int(y_min * frame.shape[0]) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
                if question:
                    # Calcular el área del rectángulo que contiene la mano
                    area = (x_max - x_min) * (y_max - y_min) * 1000
                
                    # Si el área del rectángulo es mayor que un valor umbral, reconoce el discurso
                    if area > 170:  # Ajusta este valor según tu necesidad
                        with sr.Microphone() as source:
                            # Indica las instrucciones con el sintetizador de voz
                            engine.say('Has activado Chat GPT. Realiza una breve pregunta')
                            engine.runAndWait()
                            
                            # Reconocer voz a texto
                            recognizer.adjust_for_ambient_noise(source)
                            audio = recognizer.listen(source)
                            
                            # Indica instrucciones con sintetizador de voz
                            engine.say('Procesando tu pregunta')
                            engine.runAndWait()
                        
                        # Intenta entender la pregunta
                        try:
                            pregunta = recognizer.recognize_google(audio, language="es-ES")
                        except sr.UnknownValueError:
                            print("No se pudo entender el audio")
                        except sr.RequestError as e:
                            print("Error en la solicitud: {0}".format(e))
                        
                        # Si se entiende la pregunta, se ingresa como input a chatGPT
                        if pregunta:
                            respuesta = openai.Completion.create(
                                engine="text-davinci-002",
                                prompt=pregunta,
                                max_tokens=200  # Ajusta el número de tokens según tu preferencia
                            )
                            print("Pregunta:", pregunta)
                            
                            # Guardar respuesta
                            respuesta_texto = respuesta.choices[0].text
                            
                            # Usamos el sintetizador de voz para reproducir la respuesta
                            engine.say(respuesta_texto)
                            engine.runAndWait()
                            time.sleep(1)
                            
                            # Realiza instrucciones con el sintetizador de voz
                            engine.say('Gracias. Ahora puedes volver a alejar tu mano')
                            engine.runAndWait()
                            time.sleep(1)
                    
                if number == True:
                    # Add numbers to landmarks
                    for idx, landmark in enumerate(landmarks.landmark):
                        h, w, _ = frame.shape
                        cx, cy = int(landmark.x * w), int(landmark.y * h)
                        cv2.putText(frame, str(idx), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                
                if changeColor is not None:
                    for mark in changeColor:
                        # Change the color of a specific landmark (e.g., landmark 8)
                        landmark_index_to_color = mark
                        landmark_to_change = landmarks.landmark[landmark_index_to_color]
                        cx, cy = int(landmark_to_change.x * frame.shape[1]), int(landmark_to_change.y * frame.shape[0])
                        cv2.circle(frame, (cx, cy), 5, color, -1)  # Change color to red (BGR format)
        
        # Write a instructions on screen
        if question:
            cv2.putText(frame, f'Acerca tu mano para realizar una consulta a chatGPT',
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 150, 255), 2)
        
        cv2.putText(frame, f'Saluda! Te estamos grabando :)',
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (50, 50, 255), 3)
        
        width, height = 1600, 900
        frame = cv2.resize(frame, (width, height))
        cv2.imshow('Hand Landmarks', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# ----------------------------------------------
# Use sign language recognizer from roboflow
# ----------------------------------------------
def implementation(model_directory, labels, source):
    model = YOLO(model_directory)

    model.predict(source=source, show=True)  # Generador de objetos Results
    results = model(source=source, stream=True, show = True)  # Generador de objetos Results
    
    for r in results:
        boxes = r.boxes  # Objetos Boxes para las salidas de cuadros delimitadores
        masks = r.masks  # Objetos Masks para las salidas de máscaras de segmentación
        probs = r.probs  # Probabilidades de clase para las salidas de clasificación
        
        count = 0
        if len(boxes) > 0:
            classes = int(boxes.cls[0])
            predCharacter = labels[classes]

## Usage: Hand detection

In [28]:
# Open the webcam
cap = cv2.VideoCapture(1)

In [ ]:
show = True  # True / False
number = False  # True / False
changeColor = [8,17, 4, 16] # Lista con números del 0 al 20.
color = (250, 155, 0)  # Colores BGR (Blue, Green, Red)
question = False  # True / False

handTracking(show, number, changeColor, color, question)

## Usage: Hand Gesture

In [ ]:
model_directory = r'signLenguage_Model.pt'
labels = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'F', 5: 'Rock', 6: 'U', 7: 'V', 8:'Y'}
source = '1'
implementation(model_directory, labels, source)

## Usage: Body detection

In [ ]:
import cv2
import mediapipe as mp

# Inicializa el módulo MediaPipe Pose
mp_pose = mp.solutions.pose

# Inicializa la cámara
cap = cv2.VideoCapture(1)

# Inicializa el detector de pose
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue

        # Convierte la imagen a blanco y negro
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Realiza la detección de la pose
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            # Dibuja las landmarks y líneas en la imagen
            mp_drawing = mp.solutions.drawing_utils
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Obtiene las coordenadas del bounding box
            bbox_cords = []
            for landmark in results.pose_landmarks.landmark:
                h, w, _ = frame.shape
                cx, cy = int(landmark.x * w), int(landmark.y * h)
                bbox_cords.append((cx, cy))
            min_x = min(bbox_cords, key=lambda item: item[0])[0]
            min_y = min(bbox_cords, key=lambda item: item[1])[1]
            max_x = max(bbox_cords, key=lambda item: item[0])[0]
            max_y = max(bbox_cords, key=lambda item: item[1])[1]

            # Dibuja el bounding box
            cv2.rectangle(frame, (min_x, min_y), (max_x, max_y), (0, 255, 0), 2)

        cv2.putText(frame, f'Saluda! Te estamos grabando :)',
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (50, 50, 255), 3)

        width, height = 1600, 900
        frame = cv2.resize(frame, (width, height))
        cv2.imshow('Hand Landmarks', frame)
        
        # Si se presiona la tecla 'q', se sale del bucle
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Libera la cámara y cierra las ventanas
cap.release()
cv2.destroyAllWindows()
